In [14]:
# Import required libraries
import hvplot.pandas
import pandas as pd
from config import geoapify_key
import requests

In [2]:
# turn off warning messages
import warnings
warnings.filterwarnings('ignore')

In [3]:
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()
#city_data_df.set_index('City_ID')


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2022-12-06 20:35:55,-46.1927,168.8643,52.48,55,1,8.52
1,1,Woodstock,CA,2022-12-06 20:35:55,43.1334,-80.7497,42.89,91,100,4.90
2,2,Chernyshevskiy,RU,2022-12-06 20:35:55,63.0128,112.4714,8.83,93,100,11.03
3,3,Fort Nelson,CA,2022-12-06 20:35:56,58.8053,-122.7002,-24.21,69,75,0.00
4,4,Punta Arenas,CL,2022-12-06 20:32:27,-53.1500,-70.9167,48.31,46,20,29.93


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure the map plot_1

map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
   
)

# display map_plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [6]:
# Configure the map plot_2

map_plot_2 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500
)

# display map_plot_2
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [7]:
# Configure the map plot_3
map_plot_3 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 1200,
    frame_height = 800,
    #size='Max Temp',
    scale=1,
    color='Max Temp',
    cnorm='eq_hist',
    hover=['City', 'Max Temp']
)

map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Max Temp)

In [8]:
# Configure the map plot_4
map_plot_4 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'EsriNatGeo',
    frame_width = 1200,
    frame_height = 800,
    #size='Max Temp',
    scale=1,
    color='Max Temp',
    cnorm='eq_hist',
    hover=['City', 'Max Temp']
)

map_plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Max Temp)

In [9]:
# Ask the cutomer to add a minimum and maximum temperature value.

min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

In [10]:
preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)
]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Vogan,TG,2022-12-06 20:35:57,6.3333,1.5333,82.22,84,89,9.51
6,6,Hilo,US,2022-12-06 20:34:37,19.7297,-155.0900,76.24,94,100,5.75
9,9,Atar,MR,2022-12-06 20:35:58,20.5169,-13.0499,79.86,27,0,6.31
11,11,Rikitea,PF,2022-12-06 20:34:20,-23.1203,-134.9692,75.18,76,11,15.66
12,12,Rocha,UY,2022-12-06 20:36:00,-34.4833,-54.3333,82.74,32,0,11.81


In [11]:
preferred_cities_df.count()

City_ID       166
City          166
Country       166
Date          166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
dtype: int64

In [12]:
# Creat a new dataframe called hotel_df to store hotel names along with city, country, max temp, and coordnates.
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Vogan,TG,82.22,6.3333,1.5333,
6,Hilo,US,76.24,19.7297,-155.0900,
9,Atar,MR,79.86,20.5169,-13.0499,
11,Rikitea,PF,75.18,-23.1203,-134.9692,
12,Rocha,UY,82.74,-34.4833,-54.3333,


In [37]:
# Set initial parameters to search for a hotel

radius = 5000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
}

# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latidue and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}'

    # Set up the base URL for the Geoapigy Places API
    base_url = 'https://api.geoapify.com/v2/places'

    # Make requests and retrieve the JSON data by using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    

    # Get the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
        
        

    except (KeyError, IndexError):
        # If not hotel is found, set name to 'No hotel found'
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'

    hotel = hotel_df.loc[index, 'Hotel Name']
    city = hotel_df.loc[index, 'City']
    country = hotel_df.loc[index, 'Country']

    print(f'{city}, {country} / / / / / HOTEL: {hotel}')



Vogan, TG / / / / / HOTEL: Hôtel Meduis
Hilo, US / / / / / HOTEL: Dolphin Bay Hotel
Atar, MR / / / / / HOTEL: فندق إم كا تي
Rikitea, PF / / / / / HOTEL: Chez Bianca & Benoit
Rocha, UY / / / / / HOTEL: Hotel Trocadero
Kavieng, PG / / / / / HOTEL: Kavieng Niu Lodge
Kapaa, US / / / / / HOTEL: Sheraton Kauai Coconut Beach Resort
Puerto Ayora, EC / / / / / HOTEL: Muyuyo Galápagos
Arlit, NE / / / / / HOTEL: No hotel found
Talara, PE / / / / / HOTEL: Hotel Punta Pariñas
Bengkulu, ID / / / / / HOTEL: Hotel Andalas Permai
Rio De Janeiro, BR / / / / / HOTEL: Corinto Hotel
Sao Jose Da Coroa Grande, BR / / / / / HOTEL: Hotel Praia Azul
Kyaikto, MM / / / / / HOTEL: Golden Bliss
Niquero, CU / / / / / HOTEL: Hotel Niquero
Ixtapa, MX / / / / / HOTEL: No hotel found
Georgetown, MY / / / / / HOTEL: G Hotel Kelawai
Kalianget, ID / / / / / HOTEL: No hotel found
Sinnar, SD / / / / / HOTEL: No hotel found
Atuona, PF / / / / / HOTEL: Pearl Resort
Banda Aceh, ID / / / / / HOTEL: Oasis Atjeh
Kirakira, SB / / /

In [38]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Vogan,TG,82.22,6.3333,1.5333,Hôtel Meduis
6,Hilo,US,76.24,19.7297,-155.0900,Dolphin Bay Hotel
9,Atar,MR,79.86,20.5169,-13.0499,فندق إم كا تي
11,Rikitea,PF,75.18,-23.1203,-134.9692,Chez Bianca & Benoit
12,Rocha,UY,82.74,-34.4833,-54.3333,Hotel Trocadero
...,...,...,...,...,...,...
539,Nabire,ID,75.04,-3.3667,135.4833,Hotel Anggrek
540,Piacabucu,BR,77.32,-10.4056,-36.4344,O Leão
541,Pisco,PE,75.25,-13.7000,-76.2167,Gran Hotel Chincha
542,Tecoanapa,MX,83.16,16.5167,-98.7500,No hotel found


In [40]:
# Configure the map

hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=1200,
    frame_height = 800,
    size = 'Max Temp',
    scale=1,
    color = 'City',
    hover_cols=['Hotel Name', 'Country']
)

# Display Map

hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)